In [ ]:
from copy import copy
from functools import partial

import adaptive
import numpy as np

import funcs

adaptive.notebook_extension()
%opts Image {+axiswise} [colorbar=True aspect='square'] (cmap='viridis')

In [ ]:
def conductance_rotation(x, val, syst_pars, params):
    import funcs, common

    for k in ["angle"]:
        syst_pars[k] = val[k]

    params["mu_lead"] = params["mu"]
    val["V_bias"], theta = x
    val["B_x"], val["B_y"], val["B_z"] = common.spherical_coords(params["B"], theta, 90)

    params = funcs.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val["V_bias"])


gaps = {
    -10: {0: 193.625, 45: 113.125},
    -8: {0: 165.875, 45: 96.625},
    -6: {0: 147.375, 45: 79.125},
    -4: {0: 119.875, 45: 63.125},
    -2: {0: 98.625, 45: 46.625},
    0: {0: 78.125, 45: 32.125},
    2: {0: 59.875, 45: 20.875},
    4: {0: 39.375, 45: 13.125},
}

params = dict(
    c_tunnel=3 / 4, V_barrier=40, mu_sc=100, mu=15, B=0.25, **funcs.constants.__dict__
)

syst_pars = dict(
    a=10,
    onsite_disorder=False,
    L=2000,
    coverage_angle=135,
    r1=35,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    A_correction=True,
)

val = {"g": 50, "alpha": 20, "orbital": True, "gradient": -4, "angle": 45}


params["V"] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
params["Delta"] = gaps[val["gradient"]][val["angle"]]
f = partial(
    conductance_rotation, val=val, params=copy(params), syst_pars=copy(syst_pars)
)
learner = adaptive.Learner2D(f, [(-1, 1), (-90, 135)])
fname = "thesis-cover.pickle"
learner.load(fname)

In [ ]:
learner.plot(tri_alpha=0.4)

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.npoints > 20000, ntasks=10)
saver = runner.start_periodic_saving(dict(fname=fname), 180)
runner.live_info()

In [ ]:
import itertools
import os

import adaptive
import holoviews.plotting.mpl
import matplotlib
import matplotlib.cm
import matplotlib.tri as mtri
import numpy as np
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm


def learner_till(till, learner, data):
    new_learner = adaptive.Learner2D(None, bounds=learner.bounds)
    new_learner.data = {k: v for k, v in data[:till]}
    for x, y in learner._bounds_points:
        # always include the bounds
        new_learner.tell((x, y), learner.data[x, y])
    return new_learner


def plot_tri(learner, ax, xy_size):
    ip = learner.ip()
    tri = ip.tri
    xs, ys = tri.points.T
    x_size, y_size = xy_size
    triang = mtri.Triangulation(x_size * xs, y_size * ys, triangles=tri.vertices, )
    return ax.triplot(triang, c="k", lw=0.3, alpha=1, zorder=4), (ip.values, triang)


def to_gradient(data, flip=False):
    n, m = data.shape if flip else data.shape[::-1]
    x = np.linspace(1, 0, n)
    mu = 0.5  # transition
    x = 1 / (np.exp((x - mu) * 20) + 1)  # Fermi-Dirac like
    gradient = x.reshape(1, -1).repeat(m, 0)
    if not flip:
        gradient = gradient.T
#     gradient_rgb = matplotlib.cm.viridis_r(data)
#     gradient_rgb = matplotlib.cm.gist_heat_r(data)
    gradient_rgb = matplotlib.cm.inferno(data)
    gradient_rgb[:, :, -1] = gradient
    return gradient_rgb


def get_new_artists(npoints, learner, data, ax, xy_size):
    new_learner = learner_till(npoints, learner, data)
    (line1, line2), (zs, triang) = plot_tri(new_learner, ax, xy_size)
    
    data = learner.plot(n=400).Image.I.data  # This uses the original learner!
    x_size, y_size = xy_size
    im = ax.imshow(
        to_gradient(data),
        extent=(-0.5 * x_size, 0.5 * x_size, -0.5 * y_size, 0.5 * y_size),
        zorder=3
    )
    print(zs.shape)
    ax.tripcolor(triang, zs.flatten(), zorder=0, cmap='inferno')
    return im, line1, line2


data = list(learner.data.items())

# measured from the guides in Tomas's thesis: `thesis_cover.pdf`
x_right = 14.335
x_left = 0.591
y_top = 0.591
y_bottom = 10.039

inch_per_cm = 2.54
margin = 0.5 / inch_per_cm  # add 5 mm margin on each side

x_size = x_right - x_left + 2 * margin
y_size = y_bottom - y_top + 2 * margin
xy_size = x_size, y_size

spine_mid = x_size / 2
spine_size = 0.8 / inch_per_cm

fig, ax = plt.subplots(figsize=(x_size, y_size))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

ax.set_xticks([])
ax.set_yticks([])

im, line1, line2 = get_new_artists(len(data)//5, learner, data, ax, xy_size)


title = "Towards realistic numerical simulations \n of Majorana devices"
title2 = "Towards realistic numerical simulations of Majorana devices"
author = "Bas Nijholt"

text_color = "white"

ax.axis("off")
text_zorder = 4
for pos, text in zip([-0.6, 0.7], [author, title]):
    ax.text(
        x_size / 4,
        pos * (y_size - margin) / 2,
        text,
        color=text_color,
        weight="bold",
        verticalalignment="center",
        horizontalalignment="center",
        fontsize=18,
        zorder=text_zorder,
    )

ax.text(
    -0.09,
    y_size / 4 - 0.9,
    title2,
    color=text_color,
    weight="bold",
    fontsize=10,
    zorder=text_zorder,
    rotation=-90,
    verticalalignment="center",
    horizontalalignment="left",
)
ax.text(
    -0.09,
    -y_size / 4 - 1,
    author,
    fontsize=10,
    zorder=text_zorder,
    color=text_color,
    weight="bold",
    rotation=-90,
    verticalalignment="center",
    horizontalalignment="left",
)

for i in [-1, +1]:
    ax.vlines(i * spine_size / 2, -y_size / 2, y_size / 2, linestyles=":", color="cyan")
    ax.vlines(
        -i * x_size / 2 + i * margin,
        -y_size / 2,
        y_size / 2,
        linestyles=":",
        color="cyan",
    )
    ax.hlines(
        -i * y_size / 2 + i * margin,
        -x_size / 2,
        x_size / 2,
        linestyles=":",
        color="cyan",
    )

ax.set_xlim(-x_size / 2, x_size / 2)
ax.set_ylim(-y_size / 2, y_size / 2)
fig.savefig("thesis-cover.pdf", format="pdf", bbox_inches="tight", pad_inches=0.001, dpi=500)

## animation 

In [ ]:
N = 128
cycle = itertools.cycle(range(N))


def get_cmap(roll, cmap=matplotlib.cm.inferno):
    import matplotlib.colors as mcolors

    to_roll = (
        np.linspace(0, 1, N // 2).tolist()[:-1]
        + np.linspace(0, 1, N // 2).tolist()[::-1]
    )
    colors = cmap(np.roll(to_roll, roll))
    rolled_cmap = mcolors.LinearSegmentedColormap.from_list("rolled_cmap", colors)
    return rolled_cmap

nseconds = 15
npoints = (len(data) * np.linspace(0, 1, 24 * nseconds) ** 2).astype(int)
artists = [get_new_artists(n, learner, data) for n in tqdm(npoints)]

ani = animation.ArtistAnimation(fig, artists, blit=True)

metadata = dict(
    artist="Bas Nijholt",
    title="Learning a Majorana phase diagram",
    genre="Quantum computing",
    subject="python-adaptive/adaptive",
    copyright="Bas Nijholt",
    comment="see http://adaptive.readthedocs.io",
)

writer = FFMpegWriter(fps=24, metadata=metadata, bitrate=2 * 1800)
ani.save("movie.mp4", writer=writer, dpi=200)